In [1]:
# Import our data processing library (note: you may have to install this!)


import pandas as pd
import altair as alt


# Let's use this to upload a sample dataset and show the start of the dataset
house= pd.read_csv("1553768847-housing.csv")
house.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


In [2]:
sample = house.sample(n=1000)

sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 4949 to 10355
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           1000 non-null   float64
 1   latitude            1000 non-null   float64
 2   housing_median_age  1000 non-null   int64  
 3   total_rooms         1000 non-null   int64  
 4   total_bedrooms      989 non-null    float64
 5   population          1000 non-null   int64  
 6   households          1000 non-null   int64  
 7   median_income       1000 non-null   float64
 8   ocean_proximity     1000 non-null   object 
 9   median_house_value  1000 non-null   int64  
dtypes: float64(4), int64(5), object(1)
memory usage: 85.9+ KB


In [3]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 112.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 93.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 104.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 39.7 MB/s  eta 0:00:01
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import geopandas as gpd


In [5]:
gdf = gpd.read_file(
    "https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/california-counties.geojson"
)


geojson = gdf.loc[:, ["name", "geometry"]].__geo_interface__
california = alt.Data(values=geojson["features"])
back_map = alt.Chart(california).mark_geoshape(
            stroke="lightgray", fill = "lightgray").project(
        type="mercator"    )


In [18]:

blocks = alt.Chart(sample).mark_circle().encode(
    longitude='longitude',
    latitude='latitude',
    size=alt.value(10),
    color = "ocean_proximity"
)
back_map+blocks

alt.LayerChart(...)

In [11]:
selection = alt.selection(type="interval", encodings = ["x","y"])
back_map = alt.Chart(california).mark_geoshape(
            stroke="lightgray", fill = "lightgray").project(
        type="mercator")
blocks = alt.Chart(sample).mark_circle().encode(
    x=alt.X('longitude', scale = alt.Scale(domain=(-130, -110))),
    y=alt.Y('latitude', scale = alt.Scale(domain=(32, 42))),
    size=alt.value(10),
    color = alt.condition(selection, "ocean_proximity", alt.value('lightgray'))
).add_selection(selection)

coords = alt.Chart(sample).mark_circle().encode(
    longitude='longitude',
    latitude='latitude',
    size=alt.value(10),
    color = alt.condition(selection, "ocean_proximity", alt.value('lightgray'))
)
fmap = (back_map+coords)

hist = alt.Chart(sample).mark_circle().encode(
    x = "population", 
    y = "median_house_value",
    color = "ocean_proximity"
).transform_filter(selection)

blocks | fmap | hist

alt.HConcatChart(...)

In [27]:
selection = alt.selection(type="interval", encodings = ["x","y"])
back_map = alt.Chart(california).mark_geoshape(
            stroke="lightgray", fill = "lightgray").project(
        type="mercator")
blocks = alt.Chart(sample).mark_circle().encode(
    x=alt.X('longitude', scale = alt.Scale(domain=(-130, -110)), title = "Longitude"),
    y=alt.Y('latitude', scale = alt.Scale(domain=(32, 42)), title = "Latitude"),
    size=alt.value(10),
    color = alt.condition(selection, "ocean_proximity", alt.value('lightgray'))
).add_selection(selection).interactive()

coords = alt.Chart(sample).mark_circle().encode(
    longitude='longitude',
    latitude='latitude',
    size=alt.value(10),
    color = alt.condition(selection, "ocean_proximity", alt.value('lightgray'))
)
fmap = (back_map+coords)


scat1 = alt.Chart(sample).mark_circle().encode(
    x = "population", 
    y = "median_house_value",
    color = "ocean_proximity"
).transform_filter(selection)

scat2 = alt.Chart(sample).mark_circle().encode(
    x = "total_bedrooms", 
    y = "median_house_value",
    color = "ocean_proximity"
).transform_filter(selection)

scat3 = alt.Chart(sample).mark_circle().encode(
    x = alt.X("median_income", title = "Median Income"), 
    y = alt.Y("median_house_value", title = "Median Home Value"),
    color = alt.Color("ocean_proximity", title = "Ocean Proximity")
).transform_filter(selection)


blocks | fmap | scat3 #| scat2 | scat1

alt.HConcatChart(...)

In [25]:
alt.Chart(sample).mark_circle().encode(
    x = "median_income", 
    y = "median_house_value",
    color = "ocean_proximity"
)

alt.Chart(...)